In [2]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload

from instruments.srs.srsdg645 import SRSDG645
from quantities import ns
from quantities import Quantity

In [4]:
DG645_ADDRESS = "130.102.202.2"
DG645_BARE_SOCKET_PORT = 5025
inst = SRSDG645.open_tcpip(DG645_ADDRESS, DG645_BARE_SOCKET_PORT)

In [23]:
timing_delay = (
            float(inst.channel["C"].delay[1].to("ns").magnitude)
        )  # ns

In [14]:
print(type(inst.channel["C"].delay[1]))

<class 'pint.util.Quantity'>


In [22]:
inst.channel["C"].delay[1].to("ns").magnitude

1199.9999999999998